---------
**Block 1: Import Libraries & Setup Authentication**
- **Purpose:** Initialize libraries and set up the environment.
- **Input:** User's environment choice (Colab or local Jupyter).
- **Output:** Authenticated environment with required packages imported and, if applicable, Google Drive mounted.


In [ ]:
# -----------------------------------------------------------------------------
# Block 1: Import all libraries and Setup Authentication & Environment
# -----------------------------------------------------------------------------

import geemap
import os

# ---------------------------------------------------------------------------
# Ask user if they are working in Google Colab or local Jupyter
# ---------------------------------------------------------------------------
env_choice = input("\nAre you working in Google Colab or local Jupyter? (colab/jupyter): ").strip().lower()
if env_choice == "colab":
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        print("Google Drive mounted successfully.")
    except Exception as e:
        print("Error mounting Google Drive:", e)
    # Install necessary packages in Colab if not already installed.
    try:
        !pip install leafmap localtileserver
    except Exception as e:
        print("Error installing required packages:", e)
else:
    print("Proceeding with local Jupyter environment.")

import leafmap
import localtileserver

print("\n===============================================")
print("= Block 1: Setup and Authentication completed =")
print("===============================================")

------------------
**Block 2: Visualize Base Map for ROI**
- **Purpose:** Display an interactive satellite basemap for drawing a region of interest (ROI).
- **Input:** Map center coordinates, zoom level, and display dimensions.
- **Output:** A map with a satellite imagery basemap ready for ROI selection.
- **Note:** Draw polygon with features on the left side of the window.

In [ ]:
# -----------------------------------------------------------------------------
# Block 2: Visualize Google Base Map to Draw Polygon on ROI
# -----------------------------------------------------------------------------

m = leafmap.Map(center=[38.5449, -121.7405], zoom=12, height="800px")
m.add_basemap("SATELLITE")
m

--------------
**Block 3: Save Tiff File of ROI**
- **Purpose:** Capture a user-defined bounding box (ROI) and save it as a GeoTIFF file.
- **Input:** User-drawn bounding box and a Google Drive base directory provided by the user.
- **Output:** A GeoTIFF file saved in an `EarthEngineDownload` folder within the specified Google Drive directory.

In [ ]:
# -----------------------------------------------------------------------------
# Block 3: Saving Tiff File of ROI
# -----------------------------------------------------------------------------

#Saving drawn bounding box
bbox = m.user_roi_bounds()

# Prompt the user to input the base directory in Google Drive
base_dir = input("Enter your Google Drive base directory (e.g., '/content/drive/MyDrive'): ")

# Define the download folder path
download_folder = os.path.join(base_dir, "EarthEngineDownload")

# Create the folder if it does not exist
if not os.path.exists(download_folder):
    os.makedirs(download_folder)
    print(f"Created folder: {download_folder}")
else:
    print(f"Folder already exists: {download_folder}")

# Define the output file path within the EarthEngineDownload folder
output_path = os.path.join(download_folder, "Google_Map_Image.tif")

# Use leafmap to download the satellite tiles as a GeoTIFF image
leafmap.map_tiles_to_geotiff(
    output=output_path,
    bbox=bbox,
    zoom=19,
    source="Satellite",
    overwrite=True
)

print(f"GeoTIFF saved to: {output_path}")

---------
**Block 4: Display the Downloaded Tiff on Map**
- **Purpose:** Overlay the downloaded GeoTIFF image onto the map and hide the original basemap.
- **Input:** File path to the saved GeoTIFF image.
- **Output:** An updated map displaying only the GeoTIFF image layer.

In [ ]:
# -----------------------------------------------------------------------------
# Block 4: Show the Final Tiff File on Map
# -----------------------------------------------------------------------------

m.layers[-1].visible = False
m.add_raster(output_path, layer_name="Image")
m